In [11]:
import pandas as pd

In [12]:
data = {
    'image_id': ['img_001', 'img_002', 'img_003', 'img_004', 'img_005'],
    'pred': ['cat', 'dog', 'bird', 'cat', 'dog'],
    'votes_a': [10, 15, 7, 20, 5],
    'votes_b': [5, 10, 3, 8, 2],
    'votes_c': [2, 1, 0, 3, 1]
}

pd.DataFrame(data).set_index('image_id').to_csv("test_data.csv")

In [15]:
import subprocess

def add_a_vote(col, image_id):
    df = pd.read_csv("test_data.csv", index_col = "image_id")
    df.at[image_id,col] += 1
    df.to_csv("test_data.csv")
    subprocess.run(["git", "pull"])
    subprocess.run(['git', 'add', "test_data.csv"], check=True)
    subprocess.run(['git', 'commit', '-m', "update vote"], check=True)
    subprocess.run(["git","push"])
    
    

In [ ]:
add_a_vote("votes_a","img_001")